## Imports

In [1]:
!echo $CUDA_VISIBLE_DEVICES

2,3


In [2]:
import torch
from torch import optim
from ignite.handlers import Checkpoint

In [6]:
%run -n ../train_classification.py

In [8]:
%run ../models/classification/__init__.py
%run ../models/checkpoint/__init__.py
%run ../datasets/__init__.py
%run ../utils/__init__.py

In [8]:
# DEVICE = torch.device('cpu')
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

## Load stuff

### Load datasets

In [22]:
dataset_name = 'covid-uc'
dataset_kwargs = {
    'max_samples': None,
    'batch_size': 26,
    'image_size': (512, 512),
}
train_kwargs = {
    'augment': True,
    # 'augment_label': 'covid',
    'oversample': True,
    'oversample_label': 'covid',
    'oversample_max_ratio': 10,
}

train_dataloader = prepare_data_classification(dataset_name, 'train',
                                               **dataset_kwargs, **train_kwargs)
val_dataloader = prepare_data_classification(dataset_name, 'val', **dataset_kwargs)
len(train_dataloader.dataset)

Loading covid-uc/train dataset...
	Augmenting all samples:  new-total=4770 original=795
	Oversampling covid: ratio=10 positives=138 negatives=4632 new-total=6012 original=4770
Loading covid-uc/val dataset...


4770

### Load pretrained model

In [14]:
multiple_gpu = True

In [15]:
# run_name = '0704_005511_covid-kaggle_tfs-small_lr1e-06'
# run_name = '0714_232500_cxr14_densenet-121_lr1e-06'
# run_name = '0714_232518_cxr14_densenet-121_lr1e-06'
# run_name = '0716_133211_cxr14_densenet-121_lr1e-06_us_aug-0_Pneumonia'
run_name = '0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid'
compiled_model = load_compiled_model_classification(run_name,
                                                    debug=False,
                                                    multiple_gpu=False,
                                                    device=DEVICE)

In [12]:
compiled_model.metadata

{'model_kwargs': {'model_name': 'densenet-121',
  'labels': ['covid', 'pneumonia', 'normal'],
  'imagenet': True,
  'freeze': False,
  'image_size': [512, 512]},
 'opt_kwargs': {'lr': 1e-06},
 'hparams': {'loss_name': 'cross-entropy',
  'loss_kwargs': {},
  'batch_size': 28},
 'dataset_kwargs': {'dataset_name': 'covid-x',
  'labels': None,
  'max_samples': None,
  'batch_size': 28,
  'image_size': [512, 512]},
 'dataset_train_kwargs': {'oversample': True,
  'oversample_label': 'covid',
  'oversample_max_ratio': None,
  'augment': True,
  'augment_label': 'covid',
  'augment_kwargs': {'crop': 0.8,
   'translate': 0.1,
   'rotation': 15,
   'contrast': 0.5,
   'brightness': 0.5},
  'undersample': False,
  'undersample_label': None}}

### Use pre-trained weights new model

In [16]:
old_compiled_model = compiled_model

In [17]:
lr = 0.0001
cnn_name = 'densenet-121'
run_name = f'{get_timestamp()}_{dataset_name}_{cnn_name}_lr{lr}_os-max10_aug_pre-covid-x'
run_name

'0722_201835_covid-uc_densenet-121_lr0.0001_os-max10_aug_pre-covid-x'

In [19]:
model = init_empty_model(cnn_name,
                         train_dataloader.dataset.labels,
                         multilabel=train_dataloader.dataset.multilabel,
                         pretrained_cnn=old_compiled_model.model.base_cnn,
                         imagenet=False,
                         freeze=False,
                        ).to(DEVICE)

if multiple_gpu:
    model = nn.DataParallel(model)

optimizer = optim.Adam(model.parameters(), lr=lr)

# TODO: metadata!!
compiled_model = CompiledModel(model, optimizer)

del old_compiled_model

In [20]:
run_name

'0722_201835_covid-uc_densenet-121_lr0.0001_os-max10_aug_pre-covid-x'

### ...or create model

In [ ]:
lr = 0.000001
cnn_name = 'densenet-121'
run_name = f'{get_timestamp()}_{dataset_name}_{cnn_name}_lr{lr}'

model = init_empty_model(cnn_name,
                         train_dataloader.dataset.labels,
                         multilabel=train_dataloader.dataset.multilabel,
                         imagenet=True,
                         freeze=False,
                        ).to(DEVICE)

if multiple_gpu:
    model = nn.DataParallel(model)

optimizer = optim.Adam(model.parameters(), lr=lr)

compiled_model = CompiledModel(model, optimizer)

## Train

In [28]:
loss_name = 'cross-entropy'

In [30]:
%%time

# print_metrics = ['loss', 'acc', 'hamming']
print_metrics = ['loss', 'acc', 'prec_covid', 'recall_covid']

train_metrics, val_metrics = train_model(run_name,
                                         compiled_model,
                                         train_dataloader,
                                         val_dataloader,
                                         n_epochs=5,
                                         loss_name=loss_name,
                                         print_metrics=print_metrics,
                                         debug=False,
                                         device=DEVICE,
                                        )

Training run:  0722_201835_covid-uc_densenet-121_lr0.0001_os-max10_aug_pre-covid-x
Resuming from epoch:  5
Using loss:  cross-entropy {}
--------------------------------------------------
Training...
Finished epoch 6/10 loss 0.0023 0.1011, acc 0.9982 0.8977, prec_covid 0.9993 0.5000, recall_covid 0.9986 0.2000, 0h 9m 30s
Finished epoch 7/10 loss 0.0132 0.0891, acc 0.9988 0.9205, prec_covid 0.9993 1.0000, recall_covid 0.9993 0.2000, 0h 9m 35s
Finished epoch 8/10 loss 0.0031 0.2854, acc 0.9990 0.8182, prec_covid 1.0000 0.3333, recall_covid 1.0000 0.2000, 0h 9m 36s
Finished epoch 9/10 loss 0.0008 0.8535, acc 0.9983 0.7500, prec_covid 0.9993 1.0000, recall_covid 1.0000 0.2000, 0h 9m 37s
Finished epoch 10/10 loss 0.0028 1.0114, acc 0.9807 0.7727, prec_covid 0.9825 0.1818, recall_covid 0.9783 0.8000, 0h 9m 36s
Average time per epoch:  0h 9m 36s
--------------------------------------------------
CPU times: user 2h 36min 10s, sys: 4min 18s, total: 2h 40min 29s
Wall time: 47min 57s


In [19]:
print(output)

Training run:  0717_063232_covid-x_densenet-121_lr1e-06_os-max2_aug-0_precxr14
Using loss:  cross-entropy {}
--------------------------------------------------
Training...
Finished epoch 1/10 loss 1.6400 2.4560, acc 0.6948 0.7421, prec_covid 0.8857 0.3600, recall_covid 0.7379 0.1915, 0h 12m 21s
Finished epoch 2/10 loss 0.3639 2.1430, acc 0.8207 0.7925, prec_covid 0.9507 0.5500, recall_covid 0.8816 0.2340, 0h 11m 22s
Finished epoch 3/10 loss 0.2061 1.8972, acc 0.8559 0.8105, prec_covid 0.9665 0.7083, recall_covid 0.9082 0.3617, 0h 11m 18s
Finished epoch 4/10 loss 0.1756 1.7136, acc 0.8800 0.8148, prec_covid 0.9753 0.7083, recall_covid 0.9236 0.3617, 0h 13m 11s
Finished epoch 5/10 loss 0.1273 1.6166, acc 0.8981 0.8278, prec_covid 0.9806 0.7500, recall_covid 0.9382 0.3830, 0h 11m 17s
Finished epoch 6/10 loss 0.1155 1.5706, acc 0.9133 0.8300, prec_covid 0.9838 0.8182, recall_covid 0.9495 0.3830, 0h 11m 17s
Finished epoch 7/10 loss 0.1014 1.4694, acc 0.9289 0.8372, prec_covid 0.9894 0.8261,

In [26]:
test_dataloader = prepare_data_classification(dataset_name, 'test', **dataset_kwargs)

Loading covid-uc/test dataset...


In [31]:
dataloaders = [
    train_dataloader,
    val_dataloader,
]

In [32]:
evaluate_and_save(run_name,
                  compiled_model.model,
                  dataloaders,
                  loss_name,
                  debug=False,
                  device=DEVICE)

Evaluating model in train...
Evaluating model in val...
Saved metrics to /mnt/workspace/medical-report-generation/classification/results/0722_201835_covid-uc_densenet-121_lr0.0001_os-max10_aug_pre-covid-x/metrics.json


{'train': {'loss': 0.41062578558921814,
  'acc': 0.9374584165003327,
  'prec_covid': 0.9,
  'prec_Non-COVID': 0.7552334943639292,
  'prec_normal': 0.9809817527627859,
  'recall_covid': 0.9782608695652174,
  'recall_Non-COVID': 0.8315602836879432,
  'recall_normal': 0.938298918387414,
  'spec_covid': 0.9676165803108808,
  'spec_Non-COVID': 0.9720998531571219,
  'spec_normal': 0.9619341563786008,
  'cm': tensor([[1344,    8,   28],
          [  41,  482,   41],
          [  81,  224, 3763]])},
 'val': {'loss': 1.011396050453186,
  'acc': 0.7727272727272727,
  'prec_covid': 0.18181818181818182,
  'prec_Non-COVID': 0.0,
  'prec_normal': 0.9696969696969697,
  'recall_covid': 0.8,
  'recall_Non-COVID': 0.0,
  'recall_normal': 0.810126582278481,
  'spec_covid': 0.7831325301204819,
  'spec_Non-COVID': 1.0,
  'spec_normal': 0.7777777777777778,
  'cm': tensor([[ 4,  0,  1],
          [ 2,  0,  2],
          [13,  3, 63]])}}

## Debug stuff

### Test metrics

In [95]:
from ignite.metrics import Accuracy, Recall, Precision

In [9]:
%run ../metrics/classification/__init__.py
%run ../metrics/classification/specificity.py
%run ../metrics/classification/accuracy.py
%run ../metrics/classification/hamming.py

In [144]:
acc = MultilabelAccuracy(output_transform=_transform_remove_loss_and_round)
ham = Hamming(output_transform=_transform_remove_loss_and_round)

In [153]:
outputs = torch.tensor([[0, 1, 1],
                        [0.3, 0.7, 0.8],
                       ])
target = torch.tensor([[0, 0, 1],
                       [0, 1, 1],
                      ])

In [154]:
acc.reset()
acc.update(_transform_remove_loss_and_round((0, outputs, target)))
acc.compute()

0.6666666666666666

In [155]:
ham.reset()
ham.update(_transform_remove_loss_and_round((0, outputs, target)))
ham.compute()

0.16666666666666666

In [ ]:
sp = Specificity()
rec = Recall()
prec = Precision()

In [ ]:
fn = _get_transform_one_class(0)

In [ ]:
# outputs = torch.tensor([[1, 2, 1, 0, 0]])
# target = torch.tensor([[1, 0, 1, 1, 2]])
outputs = torch.tensor([[0, 20, -1],
                        [-40, 2, 3],
                        [17, 5, 6],
                       ])
target = torch.tensor([0, 0, 2])
outputs, target = fn((0, outputs, target))
outputs, target

In [ ]:
sp.reset()
sp.update((outputs, target))
sp.compute()

In [ ]:
rec.reset()
rec.update((outputs, target))
rec.compute().item()

In [ ]:
prec.reset()
prec.update((outputs, target))
prec.compute().item()

### Test BCE loss

In [19]:
import torch
import numpy as np
from torch.nn.functional import binary_cross_entropy

In [11]:
%run ../losses/wbce.py

In [45]:
EPS = 1e-5

In [86]:
target = torch.tensor([[1, 0, 0, 0, 0, 0],
                       [0, 0, 1, 0, 1, 0],
                       [0, 0, 0, 0, 0, 0],
                      ])
bce = WeigthedBCELoss()

In [90]:
output_o = torch.tensor([[1, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [1, 1, 1, 1, 1, 1],
                        ]).float()
bce(output_o, target)

tensor(82.8836)

In [96]:
output_o = torch.tensor([[0, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [1, 1, 1, 1, 1, 1],
                        ]).float()
bce(output_o, target)

tensor(151.9611)

In [97]:
output_o = torch.tensor([[1, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [0, 0, 0, 0, 0, 0],
                        ]).float()
bce(output_o, target)

tensor(0.0004)

In [98]:
output_o = torch.tensor([[0, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [0, 0, 0, 0, 0, 0],
                        ]).float()
bce(output_o, target)

tensor(69.0779)

In [110]:
output = output_o.clamp(min=EPS, max=1-EPS)
output

tensor([[1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 9.9999e-01, 1.0000e-05, 9.9999e-01, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05]])

In [94]:
total = np.prod(target.size())
positive = (target == 1).sum().item()
negative = total - positive
total, positive, negative

(18, 3, 15)

In [95]:
BP = total / positive
BN = total / negative
BP, BN

(6.0, 1.2)

In [85]:
target.size(), output.size()

(torch.Size([2, 3]), torch.Size([3, 6]))

In [116]:
left = (target * torch.log(output))
left

tensor([[-1.1513e+01, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -1.0014e-05, -0.0000e+00, -1.0014e-05,
         -0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00]])

In [117]:
right = (1-target) * torch.log(1-output)
right

tensor([[-0.0000e+00, -1.0014e-05, -1.0014e-05, -1.0014e-05, -1.0014e-05,
         -1.0014e-05],
        [-1.0014e-05, -1.0014e-05, -0.0000e+00, -1.0014e-05, -0.0000e+00,
         -1.0014e-05],
        [-1.0014e-05, -1.0014e-05, -1.0014e-05, -1.0014e-05, -1.0014e-05,
         -1.0014e-05]])

In [118]:
-(weights*(left + right)).sum()

tensor(69.0779)

In [99]:
weights = torch.zeros(target.size())
weights

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [109]:
weights[target == 0] = BN
weights[target == 1] = BP
weights

tensor([[6.0000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000],
        [1.2000, 1.2000, 6.0000, 1.2000, 6.0000, 1.2000],
        [1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000]])

In [119]:
binary_cross_entropy(output_o, target.float(), weight=weights, reduction='sum')

tensor(600.)

In [120]:
bce(output_o, target)

tensor(69.0779)

In [122]:
def calc_conv_output_size(input_size, padding, kernel_size, stride, dilation=0):
    value = (input_size + 2*padding - dilation * (kernel_size - 1) - 1)
    value /= (stride)
    value += 1
    return value

In [124]:
conv = nn.Conv2d(3, 32, (8, 8), stride=4)